In [ ]:
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import os
from functools import reduce 
import math
import pickle

In [ ]:
#Load dataframe HT cohort
Cases_HT_beforeOnset2013= pd.read_pickle('/home/kiwitn01/master_thesis_hypertension-complications/Cohort_Extraction/Cases_HT_beforeOnset2013.pkl')
Cases_HT_beforeOnset2013 = Cases_HT_beforeOnset2013.rename(columns={"onset_year": "HT_onset_year"})
Cases_HT_beforeOnset2013 = Cases_HT_beforeOnset2013.rename(columns={"age_in_days": "age_in_days_HT_onset"})



In [ ]:
Cases_HT_beforeOnset2013.head()

In [ ]:
#Load Dataframe for demographic data

All_Disease_Cases = pd.read_pickle('/home/kiwitn01/master_thesis_hypertension-complications/Cohort_Extraction/Complications/All3_diseases/All/Cases_HT_all3_cohorts_all.pkl')
#All_Disease_Cases = pd.read_pickle('/home/kiwitn01/master_thesis_hypertension-complications/Cohort_Extraction/Complications/Cerebrovascular_Diseases/All/Cases_HT_CR_all.pkl')
#All_Disease_Cases = pd.read_pickle('/home/kiwitn01/master_thesis_hypertension-complications/Cohort_Extraction/Complications/Heart_Diseases/All/Cases_HT_heart_disease_all.pkl')
#All_Disease_Cases = pd.read_pickle('/home/kiwitn01/master_thesis_hypertension-complications/Cohort_Extraction/Complications/Renal_Diseases/All/Cases_HT_renal_disease_all.pkl')


In [ ]:
#age in days -> age in days of disease_onset
#needed actual age_in_days (current age) -> 

All_Disease_Cases = All_Disease_Cases.rename(columns={"age_in_days": "age_in_days_complication_onset"})
All_Disease_Cases = All_Disease_Cases.rename(columns={"onset_year": "complication_onset_year"})
All_Disease_Cases['year_of_birth']= All_Disease_Cases['date_of_birth_actual'].dt.year


In [ ]:
All_Disease_Cases.head()

In [ ]:
#All_Disease_Cases = All_Disease_Cases.set_index('medical_record_number')

#Disease cases before HT_Onset 
Disease_Cases_before_HT_onset = All_Disease_Cases.loc[All_Disease_Cases['age_in_days_HT_onset'] >= All_Disease_Cases['age_in_days_complication_onset']]

#save unique mrns for those candidates that had disease before HT_onset
Disease_Cases_before_HT_onset_unique= Disease_Cases_before_HT_onset.groupby("medical_record_number").count()

### NEWLY diagnosed - First time disease only cases that have been NEWLY diagnosed with disease after HT_Onset
#index those MRNs that have been diagnosed with the disease already before their HT_onset and drop MRNs from entire disease set

indexNames= Disease_Cases_before_HT_onset_unique.index
All_Disease_Cases.drop(indexNames,inplace = True)

#rename cohort
Disease_Cases_after_HT_onset = All_Disease_Cases

#identify unique mrns that have been diagnosed for the first time with that disease and after their HT_Onset
Disease_Cases_after_HT_onset_unique = Disease_Cases_after_HT_onset.groupby("medical_record_number").count()
Disease_Cases_after_HT_onset_unique


In [ ]:
# new to add further demographical data to the cohorts

Disease_Cases_after_HT_onset = Disease_Cases_after_HT_onset.sort_values(by=['medical_record_number','age_in_days_complication_onset'])
Disease_Cases_after_HT_onset = Disease_Cases_after_HT_onset.reset_index('medical_record_number')
Disease_Cases_after_HT_onset = Disease_Cases_after_HT_onset.drop_duplicates('medical_record_number')
Disease_Cases_after_HT_onset = Disease_Cases_after_HT_onset.set_index('medical_record_number')
Disease_Cases_after_HT_onset

In [ ]:
Disease_Cases_after_HT_onset_All3 = Disease_Cases_after_HT_onset 
#Disease_Cases_after_HT_onset_CR = Disease_Cases_after_HT_onset 
#Disease_Cases_after_HT_onset_Heart = Disease_Cases_after_HT_onset 
#Disease_Cases_after_HT_onset_Renal = Disease_Cases_after_HT_onset 

In [ ]:
#save files
Disease_Cases_after_HT_onset_All3.to_pickle('/home/kiwitn01/master_thesis_hypertension-complications/Cohort_Extraction/Demographics/All3_Demographics.pkl')
#Disease_Cases_after_HT_onset_CR.to_pickle('/home/kiwitn01/master_thesis_hypertension-complications/Cohort_Extraction/Demographics/CR_Demographics.pkl')
#Disease_Cases_after_HT_onset_Heart.to_pickle('/home/kiwitn01/master_thesis_hypertension-complications/Cohort_Extraction/Demographics/Heart_Demographics.pkl')
#Disease_Cases_after_HT_onset_Renal.to_pickle('/home/kiwitn01/master_thesis_hypertension-complications/Cohort_Extraction/Demographics/Renal_Demographics.pkl')

In [ ]:
#merge all data into the 61k entire cohort
Cases_HT_beforeOnset2013

In [ ]:
Cases_HT_beforeOnset2013 = Cases_HT_beforeOnset2013[['race','gender','marital_status_code', 'HT_onset_year', 'date_of_birth_actual','year_of_birth', 'age_in_days_HT_onset']]
Cases_HT_beforeOnset2013.head()

In [ ]:
All3_demo = pd.read_pickle('/home/kiwitn01/master_thesis_hypertension-complications/Cohort_Extraction/Demographics/All3_Demographics.pkl')
CR_demo = pd.read_pickle('/home/kiwitn01/master_thesis_hypertension-complications/Cohort_Extraction/Demographics/CR_Demographics.pkl')
Heart_demo = pd.read_pickle('/home/kiwitn01/master_thesis_hypertension-complications/Cohort_Extraction/Demographics/Heart_Demographics.pkl')
Renal_demo = pd.read_pickle('/home/kiwitn01/master_thesis_hypertension-complications/Cohort_Extraction/Demographics/Renal_Demographics.pkl')



In [ ]:
All3_demo.head()

In [ ]:
All3 = All3_demo[['age_in_days_complication_onset','complication_onset_year', 'context_name', 'context_diagnosis_code']]
CR = CR_demo[['age_in_days_complication_onset','complication_onset_year', 'context_name', 'context_diagnosis_code']]
Heart = Heart_demo[['age_in_days_complication_onset','complication_onset_year', 'context_name', 'context_diagnosis_code']]
Renal = Renal_demo[['age_in_days_complication_onset','complication_onset_year', 'context_name', 'context_diagnosis_code']]                

In [ ]:
Merge_1 = Cases_HT_beforeOnset2013.merge(All3, on ='medical_record_number', how='outer', suffixes=('', '_y'), indicator=False)
Merge_2 = Merge_1.merge(CR, on ='medical_record_number', how='outer',suffixes=('', '_y') ,indicator=False)
Merge_3 = Merge_2.merge(Heart, on ='medical_record_number', how='outer', suffixes=('', '_y'), indicator=False)
Final_Merge = Merge_3.merge(Renal, on ='medical_record_number', how='outer', suffixes=('', '_y'), indicator=False)


Final_Merge.drop(Final_Merge.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)




In [ ]:
Final_Merge.head()

In [ ]:
Final_Merge.loc[Final_Merge['age_in_days_complication_onset'] >0]

In [ ]:
#Final_Merge.to_pickle('/home/kiwitn01/master_thesis_hypertension-complications/Cohort_Extraction/Demographics/Final_Demographics_61k.pkl')


In [ ]:
Final_Merge = pd.read_pickle('/home/kiwitn01/master_thesis_hypertension-complications/Cohort_Extraction/Demographics/Final_Demographics_61k.pkl')
#Final_Merge= Final_Merge.set_index('medical_record_number')


In [ ]:
Final_Merge

In [ ]:
BP_timeseries_data = pd.read_pickle('/home/kiwitn01/master_thesis_hypertension-complications/Time_Series/Fetched_Timeseries/VitalSigns/vitalsign_time_series_all_BP.pkl')


In [ ]:
BP_timeseries_data.head(100)

In [ ]:
#Blood pressure feature

BP_timeseries_data = pd.read_pickle('/home/kiwitn01/master_thesis_hypertension-complications/Time_Series/Fetched_Timeseries/VitalSigns/vitalsign_time_series_all_BP.pkl')
BP_timeseries_data = BP_timeseries_data.loc[BP_timeseries_data['context_name'] == 'EPIC']
BP_timeseries_data['age_in_days_measurement'] = (BP_timeseries_data['age_in_days']) + (BP_timeseries_data['time_delta_in_days'])
BP_timeseries_data = BP_timeseries_data.set_index('medical_record_number')
BP_timeseries_data = BP_timeseries_data.rename(columns={"age_in_days": "age_in_days_HT_onset"})
BP_timeseries_data.head()
                                                                                      

In [ ]:
BP_timeseries_data.loc[BP_timeseries_data['time_delta_in_days'] < 0]

In [ ]:
BP_timeseries_data= BP_timeseries_data.set_index('medical_record_number')

In [ ]:
#merge Final Merge with BP_time_Series data

BP_data = Final_Merge.merge(BP_timeseries_data, on ='medical_record_number', how='outer', suffixes=('', '_y'), indicator=False)
BP_data.drop(BP_data.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)



In [ ]:
#save file
BP_data.to_pickle('/home/kiwitn01/master_thesis_hypertension-complications/Cohort_Extraction/Demographics/BP_and_demo_data_61k.pkl')


In [ ]:
#load file to work with
BP_data = pd.read_pickle('/home/kiwitn01/master_thesis_hypertension-complications/Cohort_Extraction/Demographics/BP_and_demo_data_61k.pkl')
BP_data

In [ ]:
#check if:
# age_in_days_measurement > age_in_days HT onset 
# age_in_days_measurement < age_in_days_complication


BP_timeseries_data = BP_data.loc[((BP_data['age_in_days_measurement']) > (BP_data['age_in_days_HT_onset'])) & ((BP_data['age_in_days_measurement']) < (BP_data['age_in_days_complication_onset']))] 
BP_timeseries_data                          
                                                                                           
    

In [ ]:
BP_timeseries_data.loc[BP_timeseries_data['time_delta_in_days'] <= 0]

In [ ]:
# create BP_timeseries_data_30_days_before_event

BP_timeseries_data['delta_btw_complication_onset_and_measurement'] = BP_timeseries_data['age_in_days_complication_onset'] - BP_timeseries_data['age_in_days_measurement'] 
BP_timeseries_data


In [ ]:
BP_timeseries_data = BP_timeseries_data.loc[BP_timeseries_data['delta_btw_complication_onset_and_measurement'] > 30]
BP_timeseries_data


In [ ]:
BP_timeseries_data_30_days_before_event = BP_timeseries_data

In [ ]:
BP_timeseries_data_30_days_before_event.loc[((BP_timeseries_data_30_days_before_event['age_in_days_measurement']) < (BP_timeseries_data_30_days_before_event['age_in_days_complication_onset']))] 



In [ ]:
#save file
#BP_timeseries_data.to_pickle('/home/kiwitn01/master_thesis_hypertension-complications/Time_Series/Fetched_Timeseries/VitalSigns/vitalsign_time_series_extra_BP.pkl')                                                 

#BP_timeseries_data_30_days_before_event.to_pickle('/home/kiwitn01/master_thesis_hypertension-complications/Time_Series/Fetched_Timeseries/VitalSigns/vitalsign_time_series_extra_BP_30_days_before_event.pkl')                                                 
BP_timeseries_data_30_days_before_event = pd.read_pickle('/home/kiwitn01/master_thesis_hypertension-complications/Time_Series/Fetched_Timeseries/VitalSigns/vitalsign_time_series_extra_BP_30_days_before_event.pkl')                                                 


In [ ]:
len (BP_timeseries_data_30_days_before_event)

In [ ]:
BP_timeseries_data_30_days_before_event.loc[BP_timeseries_data_30_days_before_event['age_in_days_measurement'] < (BP_timeseries_data_30_days_before_event['age_in_days_complication_onset'] )]



In [ ]:
#creating the aggregated dataset with extra BP data

DBP = BP_timeseries_data_30_days_before_event.loc[BP_timeseries_data_30_days_before_event['context_procedure_code'] == 'DBP']
SBP = BP_timeseries_data_30_days_before_event.loc[BP_timeseries_data_30_days_before_event['context_procedure_code'] == 'SBP']

#preparing DBP
DBP['mean_DBP_extra'] = DBP.groupby(['medical_record_number']).mean()['numeric_value']
DBP['min_DBP_extra'] = DBP.groupby(['medical_record_number']).min()['numeric_value']
DBP['max_DBP_extra'] = DBP.groupby(['medical_record_number']).max()['numeric_value']

extra_DBP = DBP.filter(['mean_DBP_extra','min_DBP_extra', 'max_DBP_extra' ], axis =1)
extra_DBP= extra_DBP.reset_index('medical_record_number')
extra_DBP= extra_DBP.drop_duplicates('medical_record_number')
extra_DBP= extra_DBP.set_index('medical_record_number')

#preparing SBP
SBP['mean_SBP_extra'] = SBP.groupby(['medical_record_number']).mean()['numeric_value']
SBP['min_SBP_extra'] = SBP.groupby(['medical_record_number']).min()['numeric_value']
SBP['max_SBP_extra'] = SBP.groupby(['medical_record_number']).max()['numeric_value']

extra_SBP = SBP.filter(['mean_SBP_extra','min_SBP_extra', 'max_SBP_extra' ], axis =1)
extra_SBP= extra_SBP.reset_index('medical_record_number')
extra_SBP= extra_SBP.drop_duplicates('medical_record_number')
extra_SBP= extra_SBP.set_index('medical_record_number')


In [ ]:
print (len (extra_DBP))
print (len (extra_SBP))

In [ ]:
extra_DBP_SBP_merge =pd.merge(extra_DBP, extra_SBP, on ='medical_record_number', how='outer')
extra_DBP_SBP_merge

In [ ]:
#merge with All_unsupervised
unsupervised = pd.read_pickle("/home/kiwitn01/master_thesis_hypertension-complications/Feature Extraction/Clean_Unsupervised_Dataframes/All_unsupervised_merged_clean_Diag_no_int.pkl")


In [ ]:
unsupervised.head()

In [ ]:
final_unsupervised_with_extra_BP= pd.merge(unsupervised,extra_DBP_SBP_merge, on ='medical_record_number', how='outer')
final_unsupervised_with_extra_BP

In [ ]:
#final_unsupervised_with_extra_BP.to_pickle("/home/kiwitn01/master_thesis_hypertension-complications/Feature Extraction/Clean_Unsupervised_Dataframes/All_unsupervised_merged_clean_Diag_no_int_with_extra_BP.pkl")
